<h1>Making the Title/Artist/Year Data Frame </h1>

In [1]:
# Cash me outside how bow dah
import requests
import requests_cache
requests_cache.install_cache("cache")

In [2]:
# Extract song info from html
from bs4 import BeautifulSoup
billboard_url = "http://www.billboard.com/articles/list/2155531/the-hot-100-all-time-top-songs"
soup = BeautifulSoup((requests.get(billboard_url)).text, "html.parser")
songs = [song.text for song in soup.find_all("h3", {"class": "list-data__title"})]

In [3]:
# Extra title, artist, and year info
import re
# We consider only the first year for a song if more than one: the 100th song's case
titles  = [str(re.sub(r"\xe2\x80\xa6", r"...", re.sub("\"", "", re.match(r"\"[\"\']?.+\"[\"\']?", song).group(0)))) for song in songs]
artists = [str(re.sub("^\"|\r|(- )|(.*\".*\" )","", re.search("\" .*\r", song).group(0)).lstrip()) for song in songs]
years   = [int(re.sub("\)|\(|,", "", re.search("(\(\d{4}\))|(\(\d{4},)", song).group(0))) for song in songs]

In [4]:
import pandas as pd 
top100songs = pd.DataFrame({'Title': titles, 'Artist': artists, 'Year': years})

<h1>Adding url and lyrics data</h1>

In [5]:
top100songs.loc[53]

Artist                                           Elton John
Title     Candle in the Wind 1997 / Something About the ...
Year                                                   1997
Name: 53, dtype: object

In [6]:
import string

def processString(s, type):
    s = re.sub("['?:!.,;&/]+", " ", s).lower().strip()
    if(type != "Artist"):
        s = re.sub("-", "",s)
    s = re.sub("\s+", "-", s)
    return(s)

def getLyrics(song, newUrl = ''):
    artist = song['Artist']
    title = song['Title']
    #Remove punctuation and change to lower case.
    artist = processString(artist, "Artist")
    title = processString(title," Title")
    lyric_url = "http://www.songlyrics.com/"+artist+"/"+title+"-lyrics/"
    if newUrl != '':
        lyric_url = newUrl
#    print(lyric_url)
    soup = BeautifulSoup((requests.get(lyric_url)).text, "html.parser")
    tmp = soup.find_all('p', {"id": "songLyricsDiv"})
    lyrics = tmp[0].get_text().encode("ascii","ignore")
    if 'Sorry, we have no' in lyrics:
        lyrics = ""
    return({'Url':lyric_url, 'Lyric': lyrics})


In [7]:
result = [getLyrics(row) for idx, row in top100songs.iterrows()]

In [8]:
#Couldn't Find
# Elton John - Candle in the wind 1997 something about the way you look tonight Lyrics
# paul-mccartney-and-michael-jackson/say-say-say-lyrics/
# jewel/you-were-meant-for-me-foolish-games-lyrics/

# weird ones: 53, 83, 60

In [9]:
notFoundUrls = ["http://www.songlyrics.com/elton-john-billy-joel/candle-in-the-wind-lyrics/", "http://www.songlyrics.com/jewel-feat-kelly-clarkson/foolish-games-lyrics/"]
result[53]['Lyric'] = getLyrics(top100songs.loc[53], newUrl = notFoundUrls[0])["Lyric"].encode("ascii","ignore")
result[83]["Lyric"] = getLyrics(top100songs.loc[83], newUrl = notFoundUrls[1])["Lyric"].encode("ascii","ignore")

In [10]:
song = top100songs.loc[60]
artist = song['Artist']
title = song['Title']
lyric_url = "http://www.metrolyrics.com/say-say-say-lyrics-paul-mccartney.html"
artist = processString(artist, "Artist")
title = processString(title," Title")
soup = BeautifulSoup((requests.get(lyric_url)).text, "html.parser")
tmp = soup.find_all('div', {"id": "lyrics-body-text"})
lyrics = tmp[0].get_text().encode("ascii","ignore")

result[60]['Lyric'] = lyrics

In [11]:
result_df = pd.DataFrame(result)
main_df = pd.concat([top100songs,result_df], axis = 1)
main_df["Rank"] = range(100, 0, -1)
del main_df["Url"]

<h1>NGRAM DOOM HELLO NGRAM THERE ONCE WAS</h1>

In [12]:
import nltk 
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import WhitespaceTokenizer
from collections import Counter

In [13]:
corpus_raw = " ".join(main_df["Lyric"])
corpus_raw = re.sub(",", "", corpus_raw)
corpus_raw = re.sub("\n", " ", corpus_raw)
corpus_raw = re.sub("\\\\", "", corpus_raw)

In [14]:
corpus_raw.lower()

'this is what you do this is what you do this is what you do this is what you do  you make me wanna leave the one i\'m with to start a new relationship with you this is what you do think about a ring and all the things that come along with you make me you make me  leave the one i\'m with start a new relationship with you this is what you do think about a ring and all the things that come along with you make me you make me  before anything became between us you were like my best friend the one i used to run and talk to when me and my girl was having problems you used to say it would be okay suggest little nice things i should do and when i go home at night and lay my head down all i seem to think about was you  and how you make me wanna leave the one i\'m with start a new relationship with you this is what you do think about a ring and all the things that come along with you make me you make me  leave the one i\'m with start a new relationship with you this is what you do think about a 

In [15]:
# def common_ngrams(token, n):
#     #pattern = r'[\d.,]+|[A-Z][.A-Z]+\b\.*|\w+|\S'
#     pattern = r"(?<![@#])\b\w+(?:'\w+)?"
#     tokenizer = WhitespaceTokenizer.tokenize(pattern)
#     return Counter(ngrams(token,3)).most_common()

In [16]:
corpus_raw_split = corpus_raw.lower().split(" ")
#corpus_raw_split = [word.lower for word in corpus_raw_split]

In [17]:
#Counter(ngrams(corpus_raw_split, 3)).most_common()
corpus_raw_split[1]

'is'

In [101]:
corpus_trigram = Counter(ngrams(corpus_raw_split, 3)).most_common()
corpus_trigram_keys = [" ".join(key_tuple) for key_tuple in [key[0] for key in corpus_trigram]]
corpus_trigram_values = [item[1] for item in corpus_trigram]
corpus_trigram = {"trigram": corpus_trigram_keys, "freq": corpus_trigram_values}

In [117]:
song_list_trigrams = [Counter(ngrams(lyric.lower().split(" "), 3)) for lyric in list(main_df["Lyric"])]

In [127]:
[" ".join(key_tuple) for key_tuple in song_list_trigrams[0].keys()][1]

"been you\nand what's"